# Introduction to Ray AI Runtime (AIR)
---
*Suggested Time to Complete: 30 minutes*

![Ray AIR](intro-to-ray-images/Ray_AIR.png)
*Figure 5*

Ray AI Runtime (AIR) is a unified set of libraries for distributed data processing, model training, tuning, reinforcement learning, and model serving, all in Python. Before we lay out each library and their unique jobs to be done, let's take a moment to motivate Ray AIR by taking a high-level view of the data science and machine learning workflow.

Developing a machine learning system is an iterative and often cyclical process that roughly touches on the following stages:
1. Business Needs: work with stakeholders to identify business requirements and align on which metric to optimize
2. Data Collection: source and collect data and labels
3. Feature Engineering: turn raw data into usable material
4. Model Training: the learning part of machine learning
5. Model Evaluation: try to improve upon your baseline model with hyperparameter tuning or more feature engineering or even a more relevant set of data
6. Deployment: deploy your solution to production and/or serve your model to the end user

Each of the five main native libraries that Ray AIR wraps tackles a specific piece of the ML specific tasks outlined above and because this abstraction layer is built on top of Ray Core, it is distributed by nature.

1. Ray Data: load and manipulate data
2. Ray Train: scales model training for popular ML frameworks
3. Ray Tune: scales experiment execution and hyperparameter tuning
4. Ray Serve: scales model serving
5. Ray RLlib: for distributed reinforment learning workloads

Let's examine each library in further detail and run through an illustrative example for each.

## Ray Data
***
Ray Datasets are the standard way to load and exchange data in Ray libraries and applications. They provide basic distributed data transformations such as map, filter, and repartition, and are compatible with a variety of file formats, data sources, and distributed frameworks.

**Datasets**: Ray Datasets are the standard way to load and exchange data in Ray AIR. In AIR, Datasets are used extensively for data loading, preprocessing, and batch inference.

**Preprocessors**: Preprocessors are primitives that can be used to transform input data into features. Preprocessors operate on Datasets, which makes them scalable and compatible with a variety of datasources and dataframe libraries.

A Preprocessor is fitted during Training, and applied at runtime in both Training and Serving on data batches in the same way. AIR comess with a collection of built-in prepreocessors, and you can also define your own with simple templates.

In [ ]:
# Create a Dataset of Python objects.
ds = ray.data.range(10000)
# -> Dataset(num_blocks=200, num_rows=10000, schema=<class 'int'>)

ds.take(5)
# -> [0, 1, 2, 3, 4]

ds.count()
# -> 10000

# Create a Dataset of Arrow records.
ds = ray.data.from_items([{"col1": i, "col2": str(i)} for i in range(10000)])
# -> Dataset(num_blocks=200, num_rows=10000, schema={col1: int64, col2: string})

ds.show(5)
# -> {'col1': 0, 'col2': '0'}
# -> {'col1': 1, 'col2': '1'}
# -> {'col1': 2, 'col2': '2'}
# -> {'col1': 3, 'col2': '3'}
# -> {'col1': 4, 'col2': '4'}

ds.schema()
# -> col1: int64
# -> col2: string

### Summary
#### Key Concepts
- Datasets
- Preprocessors

#### Key API Elements in This Section
#### Next

## Ray Train
Ray Train is a lightweight library for distributed deep learning that allows you to easily supercharge your distributed PyTorch and TensorFlow training on Ray.

**Trainer**: Trainers are wrapper classes around third-party training frameworks such as XGBoost and Pytoch. They are built to help integrate with core Ray actors (for distribution), Ray Tune, and Ray Datasets.

**Checkpoints**: The AIR trainers, tuners, and custom pretrained model generate a framework-specific Checkpoint object. Checkpoints are a common interface for models that are used across different AIR components and libraries.

**Batch Predictor**: You can take a checkpoint and do batch inference using the BatchPredictor object.

In [ ]:
import ray.train as train
from ray.train import Trainer
import torch

def train_func():
    # Setup model.
    model = torch.nn.Linear(1, 1)
    model = train.torch.prepare_model(model)
    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

    # Setup data.
    input = torch.randn(1000, 1)
    labels = input * 2
    dataset = torch.utils.data.TensorDataset(input, labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32)
    dataloader = train.torch.prepare_data_loader(dataloader)

    # Train.
    for _ in range(5):
        for X, y in dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    return model.state_dict()

trainer = Trainer(backend="torch", num_workers=4)
trainer.start()
results = trainer.run(train_func)
trainer.shutdown()

print(results)

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Tune
Ray Tune is a Python library for fast hyperparameter tuning at scale. Easily distribute your trial runs to quickly find the best hyperparameters.

**Tuner**: Tuners offer scalable hyperparameter tuning as part of Ray Tune. Tuners can work seamlessly with any Trainer but also can support arbitrary training functions.

In [ ]:
from ray import tune
 
def objective(step, alpha, beta):
   return (0.1 + alpha * step / 100)**(-1) + beta * 0.1
 
def training_function(config):
   # Hyperparameters
   alpha, beta = config["alpha"], config["beta"]
   for step in range(10):
       # Iterative training function - can be any arbitrary training procedure.
       intermediate_score = objective(step, alpha, beta)
       # Feed the score back back to Tune.
       tune.report(mean_loss=intermediate_score)
 
analysis = tune.run(
   training_function,
   config={
       "alpha": tune.grid_search([0.001, 0.01, 0.1]),
       "beta": tune.choice([1, 2, 3])
   })
 
print("Best config: ", analysis.get_best_config(
   metric="mean_loss", mode="min"))
 
# Get a dataframe for analyzing trial results.
df = analysis.results_df

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray Serve
Ray Serve lets you serve machine learning models in real-time or batch using a simple Python API. Serve individual models or create composite model pipelines, where you can independently deploy, update, and scale individual components.

**Deployments**: Deploy the model as an inference service by using Ray Serve and the `PredictorDeployment` class.

In [ ]:
import requests

from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier

from ray import serve

@serve.deployment(route_prefix="/iris")
class BoostingModel:
    def __init__(self, model):
        self.model = model
        self.label_list = iris_dataset["target_names"].tolist()

    async def __call__(self, request):
        payload = await request.json()
        print(f"Received flask request with data {payload}")

        prediction = self.model.predict([payload["vector"]])[0]
        human_name = self.label_list[prediction]
        return {"result": human_name}

if __name__ == "__main__":

    # Train model.
    iris_dataset = load_iris()
    model = GradientBoostingClassifier()
    model.fit(iris_dataset["data"], iris_dataset["target"])

    # Deploy model
    serve.run(BoostingModel.bind(model))

    # Query model
    sample_request_input = {"vector": [1.2, 1.0, 1.1, 0.9]}
    response = requests.get("http://localhost:8000/iris", json=sample_request_input)
    print(response.text)
    
    # prints
    # Result:
    # {"result": "versicolor"}

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

## Ray RLLib
RLlib is the industry-standard reinforcement larning Python frameowrk built on Ray. Designed for quick interation and a fast path to production, it includes 25+ latest algorithms that are all implemented to run at scale and in multi-agent mode.

In [ ]:
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
 
tune.run(PPOTrainer, config={
   "env": "CartPole-v0",
   "framework": "torch",
   "log_level": "INFO"
})

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Three: Ray Ecosystem
---
Up until now, we've covered Ray Core, the low-level, distributed computing framework, as well as Ray AIR, the set of high-level native libraries which include Ray Data, Train, Tune, Serve, and RLlib. In addition, Ray integrates with a growing ecosystem of the most popular Python and machine learning libraries and frameworks that you may already be familiar with.

Instead of trying to create new standards, Ray allows you to scale existing workloads by unifying tools in a common interface. This interface enables you to run tasks in a distributed way, a property most of the respective backends don't have, or not to the same extent.

For example, Ray Datasets is backed by Arrow and comes with many integrations to other frameworks, such as Spark and Dask. Ray Train and RLlib are backed by the full power of Tensorflow and PyTorch. Ray Tune supports algorithms from practically every noteable HPO tool available, including Hyperopt, Optuna, Nevergrad, Ax, SigOpt, and many others. Ray Serve can be used with frameworks such as FastAPI, gradio, and Streamlit. With ample opportunity to extend current projects as well as integrate more backends in the future, a key strength of Ray is this robust design pattern for integration which you can read more about [here](https://www.anyscale.com/blog/ray-distributed-library-patterns).

For a complete list of integrations with links, go to the [Ray Ecosystem](https://docs.ray.io/en/latest/ray-overview/ray-libraries.html#) page in the docs.

### Example: HuggingFace
### Example: XGBoost
### Example: Distributed Scikit-Learn / Joblib

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Part Four: Ray Clusters
A Ray cluster consists of nodes that are connected to each other via a network. You can program against the so-called driver, the program root, which lives on the head node. The driver can run jobs, that is a collection of tasks, that are run on the nodes in the cluster. Specifically, the individual tasks of a job are run on worker processes on worker nodes.

Ray Cluster

A Ray cluster consists of a single head node and any number of connected worker nodes. The number of worker nodes may be autoscaled with application demand as specified by your Ray cluster configuration. The head node runs the autoscaler. Users can submit jobs for execution on the Ray cluster, or can interactively use the cluster by connecting to the head node and running `ray.init`.

Head Node

Every Ray cluster has one node which is designated as the head node of the cluster. The head node is identical to other worker nodes, except that it also runs singleton processes responsible for cluster management such as the autoscaler and the Ray driver processes which run Ray jobs. Ray may schedule tasks and actor on the head node just like any other worker node, unless configured otherwise.

Worker Node

Worker nodes do not run any head node management processes, and serve only to run user code in Ray tasks and actors. They participate in distributed scheduling, as well as the storage and distribution of Ray objects in cluster memory.

Autoscaling

The Ray autoscaler is a process that runs on the head node (or as a sidecar container in the head pod if using Kubernetes). When the resource demands of the Ray workload exceed the current capacity of the cluster.

### Summary
#### Key Concepts
#### Key API Elements in This Section
#### Next

# Homework
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- [Distribute a Classical Algorithm with Ray](https://github.com/ray-project/hackathon5-algo)
    - In this excercise, go to the GitHub repo linked above for details on choosing a classic algorithm implemented in Python, editing the implementation to parallelize the work with Ray, and compare your results against the sequential implementation.


# Next Steps
---
🎉 Congratulations! You have completed your first tutorial on an Introduction to Ray! We discussed the three layers of Ray: Core, AIR, and the Ecosystem. Each library in Ray AIR (Data, Train, Tune, Serve, RLLib) and the ecosystem of integrated libraries runs on Ray Core's distributed execution engine, and with Ray Clusters, you can deploy your workloads on AWS, GCP, Azure, or on Kubernetes.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- 📖 [Ray's "Getting Started" Guides](https://docs.ray.io/en/latest/ray-overview/index.html): A collection of QuickStart Guides for every library including installation walkthrough, examples, blogs, talks, and more!
- 💻 [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- 💬 [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- 📣 [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- 🙋‍♀️ [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- 🪲 [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.